## Before Start

An initial data exploration was done to first gain some insights. The classes look balanced in the train data at about 50% each, and most reviews are of length 100-200. Even after removing stopwords, the most common word (br) is still a stopword since it is just a break in the text, and does not have any meaning. It should be added to the dictionary to be removed.

In [1]:
import pandas as pd
train_data = pd.read_csv('C:/Users/DELL/Downloads/news_fulltrain.csv', header=None)
train_data.columns=['labels','text']
train_data.head()

,labels,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [2]:
from sklearn.utils import resample
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

train_data = pd.read_csv('C:/Users/DELL/Downloads/news_fulltrain.csv', header=None)
train_data.columns=['labels','text']
train_data = train_data[(train_data['labels'] == 1) | (train_data['labels'] == 4)]
train_data['labels'].unique()

train_data_satire  =train_data[train_data['labels'] == 1]
train_data_reliable  =train_data[train_data['labels'] == 4]

train_data_satire_sampled = resample(train_data_satire,
                                     replace=True,
                                     n_samples=len(train_data_satire),
                                     random_state=42)

train_data_balanced = pd.concat([train_data_satire_sampled, train_data_reliable])
train_data_balanced['labels'] = train_data_balanced['labels'].replace({1:0, 4:1})


test_data = pd.read_csv('C:/Users/DELL/Downloads/news_balancedtest.csv')

test_data.columns = ['labels', 'text']
test_data = test_data[(test_data['labels'] == 1) | (test_data['labels'] == 4)]

test_data_balanced = test_data.copy()
test_data_balanced['labels'] = test_data_balanced['labels'].replace({1: 0, 4: 1})
test_data_balanced.head()
train_data_balanced.head()
train_data_balanced = train_data_balanced.reset_index(drop=True)
test_data_balanced = test_data_balanced.reset_index(drop=True)

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from datasets import Dataset, load_metric
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
def tokenize(data):
    tokenized_data = tokenizer(data["text"],
                     padding = "max_length",
                     truncation = True,
                     max_length=128,
                     return_tensors='pt')
    tokenized_data['labels'] = torch.tensor(data["labels"])
    return tokenized_data
# tokenized_train = train_data_balanced.map(tokenize, batched=True)
# tokenized_test = test_data_balanced.map(tokenize, batched=True)
config = AutoConfig.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels = 2)

train_dataset = Dataset.from_pandas(train_data_balanced)
test_dataset = Dataset.from_pandas(test_data_balanced)

tokenized_train = train_dataset.map(tokenize, batched=True)
tokenized_test = test_dataset.map(tokenize, batched=True)

training_args = TrainingArguments(
    output_dir="./result",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,                     
    evaluation_strategy="epoch",
    learning_rate=5e-5,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    f1 = load_metric("f1")
    precision = load_metric("precision")
    recall = load_metric("recall")
    accuracy = load_metric("accuracy")

    f1_score = f1.compute(predictions=predictions, references=labels, average='binary')
    precision_score = precision.compute(predictions=predictions, references=labels, average='binary')
    recall_score = recall.compute(predictions=predictions, references=labels, average='binary')
    accuracy_score = accuracy.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_score['accuracy'],
        "f1": f1_score['f1'],}
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1499/1499 [00:00<00:00, 3485.60 examples/s]
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [4]:
trainer.train()

  6%|▌         | 500/9018 [11:30<3:12:15,  1.35s/it]

{'loss': 0.2595, 'grad_norm': 137.7894287109375, 'learning_rate': 4.722776668884454e-05, 'epoch': 0.17}


 11%|█         | 1000/9018 [22:57<3:05:15,  1.39s/it]

{'loss': 0.1989, 'grad_norm': 0.34119951725006104, 'learning_rate': 4.445553337768907e-05, 'epoch': 0.33}


 17%|█▋        | 1500/9018 [34:37<2:56:03,  1.41s/it]

...

100%|█████████▉| 9000/9018 [5:26:43<00:39,  2.20s/it]

{'loss': 0.0274, 'grad_norm': 0.006594103295356035, 'learning_rate': 9.98003992015968e-08, 'epoch': 2.99}


100%|██████████| 9018/9018 [5:27:24<00:00,  1.88s/it]c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/precision/precision.py
You can avoid this message in future by passing the argu

{'eval_loss': 0.5538364052772522, 'eval_accuracy': 0.9039359573048699, 'eval_f1': 0.9116564417177914, 'eval_runtime': 112.8047, 'eval_samples_per_second': 13.288, 'eval_steps_per_second': 1.667, 'epoch': 3.0}
{'train_runtime': 19756.9422, 'train_samples_per_second': 3.651, 'train_steps_per_second': 0.456, 'train_loss': 0.10171069434642986, 'epoch': 3.0}


TrainOutput(global_step=9018, training_loss=0.10171069434642986, metrics={'train_runtime': 19756.9422, 'train_samples_per_second': 3.651, 'train_steps_per_second': 0.456, 'train_loss': 0.10171069434642986, 'epoch': 3.0})

In [5]:
trainer.evaluate()

100%|██████████| 188/188 [00:57<00:00,  3.81it/s]c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/precision/precision.py
You can avoid this message in future by passing the argument

{'eval_loss': 0.5538364052772522,
 'eval_accuracy': 0.9039359573048699,
 'eval_f1': 0.9116564417177914,
 'eval_runtime': 63.5333,
 'eval_samples_per_second': 23.594,
 'eval_steps_per_second': 2.959,
 'epoch': 3.0}

In [6]:
# 指定保存模型的路径
model_path = "C:/Users/DELL/Downloads/model_news"

# 保存模型
model.save_pretrained(model_path)

# 同时，保存 tokenizer 到相同的路径
tokenizer.save_pretrained(model_path)


('C:/Users/DELL/Downloads/model_news\\tokenizer_config.json',
 'C:/Users/DELL/Downloads/model_news\\special_tokens_map.json',
 'C:/Users/DELL/Downloads/model_news\\vocab.json',
 'C:/Users/DELL/Downloads/model_news\\merges.txt',
 'C:/Users/DELL/Downloads/model_news\\added_tokens.json',
 'C:/Users/DELL/Downloads/model_news\\tokenizer.json')

In [8]:
import torch
from datasets import Dataset, load_metric
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
model_path = "C:/Users/DELL/Downloads/model_news"

# 加载模型
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    f1 = load_metric("f1")
    precision = load_metric("precision")
    recall = load_metric("recall")
    accuracy = load_metric("accuracy")

    f1_score = f1.compute(predictions=predictions, references=labels, average='binary')
    precision_score = precision.compute(predictions=predictions, references=labels, average='binary')
    recall_score = recall.compute(predictions=predictions, references=labels, average='binary')
    accuracy_score = accuracy.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_score['accuracy'],
        "f1": f1_score['f1'],
        "precision": precision_score['precision'],
        "recall": recall_score['recall']
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

trainer.evaluate()

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
100%|██████████| 188/188 [01:00<00:00,  3.57it/s]c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Pass

{'eval_loss': 0.5538364052772522,
 'eval_accuracy': 0.9039359573048699,
 'eval_f1': 0.9116564417177914,
 'eval_precision': 0.8443181818181819,
 'eval_recall': 0.9906666666666667,
 'eval_runtime': 64.949,
 'eval_samples_per_second': 23.08,
 'eval_steps_per_second': 2.895}

In [34]:
test_string_satire = '''
When so many actors seem content to churn out performances for a quick paycheck, a performer who adheres to his principles really stands out. Thats why Jeff Bridges made waves this week when he announced that from now on, he will only perform nude scenes. In an interview in this months GQ, the Big Lebowski star made it clear that he was more than ready to move on to a new phase in his career, leaving his clothed roles in the past. Ive been there and Ive done that, said Bridges, rattling off a laundry list of the films hes appeared in covered up. Now, I can finally afford to only take on roles that excite me. Right now, those are roles with nude scenes. Why waste my time with anything else? Powerful. Though he made it clear that he doesnt regret his previous non-nude roles, Jeff admitted that hed always struggled with pressure from directors and studios to stay clothed on camera. No more towels; no more bathrobes; no more carefully placed plants, he added. Even if my character isnt written as nude, any director I work with will have to figure out how to make him that way. Itll be a challenge for both of us, and one I cant wait to tackle. For their part, Jeffs fans have been nothing but supportive. Wow! Whether or not you agree with him, youve got to have respect for a Hollywood star with that much conviction. You keep doing you, Jeff!
'''

test_string_reliable = '''
The Alberta province health minister wants to know if swine flu shots were 'inappropriately diverted' to the Calgary Flames while thousands had to stand in line for hours for the vaccine. Alberta Health Minister Ron Liepert says he doesn't know where the NHL team got the vaccine, adding that Alberta Health Services is the only supplier in the province. Team president Ken King says the club contacted the department and asked for the clinic. Health officials have begun an investigation into the special clinic, which was held for the players and their families last Friday. Liepert says the vaccine would be diverted only with the approval of the chief medical officer of health, but he doesn't know if that was the case. Alberta's opposition parties say professional ice hockey players shouldn't be getting the vaccine ahead of cancer patients and pregnant women.
'''
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=1  # 设置为1因为我们只预测一个样本
)
trainer = Trainer(model=model, args=training_args)
print("Satire")
satire_input_x = pd.DataFrame({'labels':[0], 'text': test_string_satire})

reliable_input_x = pd.DataFrame({'labels':[0], 'text': test_string_reliable})

satire_dataset = Dataset.from_pandas(satire_input_x)
reliable_dataset = Dataset.from_pandas(reliable_input_x)

satire_input_x = satire_dataset.map(tokenize, batched=True)
reliable_input_x = reliable_dataset.map(tokenize, batched=True)

# satire_input_x['labels'] = torch.tensor(0)
# Check the number of tokens
# print("Input IDs shape:", satire_input_x['input_ids'].shape)  # 应显示 (1, 128) 或其他类似批处理形状
# print("Input IDs shape:", tokenized_test['input_ids'].shape) 
predictions = trainer.predict(satire_input_x)


logits = predictions.predictions
probs = torch.softmax(torch.tensor(logits), dim=-1)
predictions = torch.argmax(probs, dim=-1)

print("Probabilities:", probs)
print("Predictions:", predictions)

print("Reliable")

predictions = trainer.predict(reliable_input_x)


logits = predictions.predictions
probs = torch.softmax(torch.tensor(logits), dim=-1)
predictions = torch.argmax(probs, dim=-1)

print("Probabilities:", probs)
print("Predictions:", predictions)

Satire
   labels                                               text
0       0  \nWhen so many actors seem content to churn ou...
   labels                                               text
0       0  \nThe Alberta province health minister wants t...


/Users/kiriharari/miniconda3/envs/dl/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Probabilities: tensor([[0.9887, 0.0113]])
Predictions: tensor([0])
Reliable


  0%|          | 0/1 [00:00<?, ?it/s]

Probabilities: tensor([[3.0635e-05, 9.9997e-01]])
Predictions: tensor([1])
